In [13]:
import By
##CHROME SERVICE UPDATE##
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
# 
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.close()


In [12]:
##CHROMEDRIVER SERVICE CHECK##

from selenium import webdriver
import time
 
driver = webdriver.Chrome()
driver.get("https://google.com")
time.sleep(5)
driver.close()

In [17]:
##iterable part##
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Step 1: Open Google Scholar and search for the article
query = "Naturalistic physical adversarial patch for object detectors"
driver = webdriver.Chrome()
driver.get("https://scholar.google.com")
time.sleep(2) # Let the user actually see something!
search_box = driver.find_element(By.NAME,'q')
time.sleep(2) # Let the user actually see something!
search_box.send_keys(query)
time.sleep(2) # Let the user actually see something!
search_box.send_keys(Keys.RETURN)

# Step 2: Find the ".pdf" attribute in search results
pdf_link = driver.find_element(By.CSS_SELECTOR,'a[href$=".pdf"]')     
#.find_elements_by_css_selector('a[href$=".pdf"]')

# Step 3: Print the found PDF links
print(pdf_link.get_attribute("href"))

# Close the browser
driver.quit()


https://openaccess.thecvf.com/content/ICCV2021/papers/Hu_Naturalistic_Physical_Adversarial_Patch_for_Object_Detectors_ICCV_2021_paper.pdf
